In [1]:
library("DBI")
library("RPostgreSQL")
library("dbplyr")
library("dplyr")

con <- DBI::dbConnect(
    RPostgreSQL::PostgreSQL(),
    host = "localhost",
    port = 5432,
    dbname = "knock100",
    user = "guest",
    password = "guest"
)

customer_tbl <- dplyr::tbl(con, "customer")
category_tbl <- dplyr::tbl(con, "category")
product_tbl <- dplyr::tbl(con, "product")
receipt_tbl <- dplyr::tbl(con, "receipt")
store_tbl <- dplyr::tbl(con, "store")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:dbplyr’:

    ident, sql


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




### R-061:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(log_amount = log10(amount)) %>%
head

### R-062:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE )) %>%
mutate(log_amount = log(amount)) %>%
head

### R-063:
商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [ ]:
product <- tbl(con, 'product')
product %>%
transmute(product_cd, unit_price, unit_cost, unit_profit = unit_price - unit_cost) %>%
head

### R-064:
商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(avg_profit_rate = mean(unit_price - unit_cost, na.rm = TRUE)/mean(unit_price)) %>%
summarise(avg_profit_rate = min(avg_profit_rate))

### R-065:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(target_price = trunc(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

### R-066:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(target_price = round(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

### R-067:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(target_price = ceiling(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

### R-068:
商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [ ]:
product %>%
mutate(price_taxin = trunc(1.1 * unit_price)) %>%
select(unit_price, price_taxin) %>%
head

### R-069:
レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [ ]:
receipt_tbl %>%
inner_join(product_tbl, by = 'product_cd') %>% 
group_by(customer_id) %>%
summarise(
	amount_canned = sum(if_else(category_major_cd == '07',amount, 0), na.rm = TRUE),
	amount_total = sum(amount)
) %>%
filter(amount_canned > 0) %>%
mutate(canned_ratio = as.numeric(amount_canned) / amount_total) %>%
head(10)

### R-070:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [4]:
customer_tbl %>%
    inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by = "customer_id") %>%
    select(customer_id, sales_ymd, application_date) %>%
    collect() %>%
    mutate(elapsed_days = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd)) %/% lubridate::days(1)) %>%
    head()

customer_id,sales_ymd,application_date,elapsed_days
<chr>,<int>,<chr>,<dbl>
CS006214000001,20181103,20150201,1371
CS008415000097,20181118,20150322,1337
CS028414000014,20170712,20150711,732
CS025415000050,20180821,20160131,933
CS003515000195,20190605,20150306,1552
CS024514000042,20181205,20151010,1152


In [2]:
customer_tbl %>%
    inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by = "customer_id") %>%
    select(customer_id, sales_ymd, application_date) %>%
    collect() %>%
    mutate(interval = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd))) %>%
    head()

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”


customer_id,sales_ymd,application_date,interval
<chr>,<int>,<chr>,<Interval>
CS006214000001,20181103,20150201,2015-02-01 UTC--2018-11-03 UTC
CS008415000097,20181118,20150322,2015-03-22 UTC--2018-11-18 UTC
CS028414000014,20170712,20150711,2015-07-11 UTC--2017-07-12 UTC
CS025415000050,20180821,20160131,2016-01-31 UTC--2018-08-21 UTC
CS003515000195,20190605,20150306,2015-03-06 UTC--2019-06-05 UTC
CS024514000042,20181205,20151010,2015-10-10 UTC--2018-12-05 UTC


### R-071:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
# work in local
library(lubridate)

compare_date_local <- cul %>%
select(customer_id, application_date) %>%
inner_join(select(rel, customer_id, sales_ymd), by='customer_id') %>%
transmute(customer_id, sales_ymd = ymd(sales_ymd), application_date = ymd(application_date))

compare_date_local %>% 
mutate(elapsed_months = interval(application_date, sales_ymd) %/% months(1)) %>%
head

### R-072:
レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
customer_tbl %>%
inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by='customer_id') %>%
select(customer_id, sales_ymd, application_date) %>%
collect() %>%
mutate(elapsed_years = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd)) %/% months(1)) %>%
head

In [ ]:
# work in local
compare_date_local %>%
mutate(elapsed_years = interval(application_date, sales_ymd) %/% years(1)) %>%
head

### R-073:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
# work in local
compare_date_local %>%
mutate(elapsed_epoch = (sales_ymd - application_date) %>% as.numeric(units = 'secs')) %>%
head

In [ ]:
customer_tbl %>%
    inner_join(receipt_tbl %>% select(customer_id, sales_ymd), by = "customer_id") %>%
    select(customer_id, sales_ymd, application_date) %>%
    collect() %>%
    mutate(elapsed_months = lubridate::interval(lubridate::ymd(application_date), lubridate::ymd(sales_ymd)) %/% seconds(1)) %>%
    head()

### R-074:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
receipt_tbl %>%
select(sales_ymd) %>%
collect() %>%
mutate(
	elapsed_weekdays = lubridate::ymd(sales_ymd) %>% lubridate::wday(week_start = 1) -1,
	last_monday = lubridate::ymd(sales_ymd) - elapsed_weekdays
	) %>%
	head

In [ ]:
library(lubridate)
re %>%
data.frame %>%
mutate(
	sales_ymd_d = sales_ymd %>% ymd,
	elapsed_weekdays = sales_ymd_d %>% wday(week_start = 1)-1,
	last_monday = sales_ymd_d - elapsed_weekdays
	) %>%
	head

### R-075:
顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [67]:
customer_tbl %>% slice_sample(n=customer_tbl %>% count() %>% collect() %>% as.numeric() %/% 100) %>% head

ERROR: [1m[33mError[39m in [1m[1m`slice_sample()`:[22m
[33m![39m Sampling with replacement is not supported on database backends


### R-076:
顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [73]:
sampled_customer <- customer_tbl %>% 
	group_by(gender_cd) %>%
	collect() %>%
	dplyr::slice_sample(prop = 0.1)
sampled_customer %>% nrow() %>% as.numeric()

[1] 2196

### R-077:
レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。


In [78]:
receipt_tbl %>%
group_by(customer_id) %>%
summarise(log_amount = log(sum(amount, na.rm = TRUE))) %>%
mutate(mean = mean(log_amount, na.rm = TRUE), sd = sd(log_amount,na.rm = TRUE)) %>%
filter(!(log_amount <= mean+3*sd)) %>%
head

# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    log_amount  mean    sd
  <chr>               <dbl> <dbl> <dbl>
1 ZZ000000000000       16.3  7.27  1.14

### R-078:
レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [118]:
# assume you are on a local machine
library(stringr)

receipt_tbl %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
collect() %>%
filter(
	amount %>%
	{. > quantile(., probs = 0.25) + 1.5*IQR(.) | . < quantile(., probs = 0.25) - 1.5*IQR(.)}
) %>% head

customer_id,amount
<chr>,<dbl>
CS007515000119,7157
CS026414000014,6671
CS016414000063,6207
CS039814000011,8031
CS013415000226,8362
CS009415000117,8229


In [112]:
# assume you are on a local machine
library(stringr)

receipt_tbl %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
collect() %>%
filter(
	amount %>%
	{. > quantile(., probs = 0.25) + 1.5*IQR(.) | . < quantile(., probs = 0.25) - 1.5*IQR(.)}
) %>% head

customer_id,amount
<chr>,<dbl>
CS007515000119,7157
CS026414000014,6671
CS016414000063,6207
CS039814000011,8031
CS013415000226,8362
CS009415000117,8229


In [ ]:
# assume you are on a local machine
rel <- DBI::dbReadTable(con, "receipt")
library(stringr)

rel %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(iqr = quantile(amount)[4] - quantile(amount)[2]) %>%
mutate(thre_lower = quantile(amount)[2] - 1.5*iqr, thre_upper = quantile(amount)[4] + 1.5*iqr) %>%
filter(amount < thre_lower | thre_upper < amount) %>%
head

### R-079:
商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [64]:
product_tbl %>% filter(if_any(-c(), ~ is.na(.))) %>% collect() %>% 
summarise(across(everything(), ~ is.na(.) %>% sum()))

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,7,7


### R-080:
商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [78]:
product_tbl %>% filter(if_all(-c(), ~ !is.na(.))) %>% head

# Source:   lazy query [?? x 6]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  product_cd category_major_cd category_medium_cd category_small_cd unit_price
  <chr>      <chr>             <chr>              <chr>                  <int>
1 P040101001 04                0401               040101                   198
2 P040101002 04                0401               040101                   218
3 P040101003 04                0401               040101                   230
4 P040101004 04                0401               040101                   248
5 P040101005 04                0401               040101                   268
6 P040101006 04                0401               040101                   298
# … with 1 more variable: unit_cost <int>

In [75]:
# complete.cases does not works on tbl
product_tbl %>% collect() %>%
filter(complete.cases(.)) %>% nrow()

[1] 10023

### R-081:
単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
library(tidymodels)

pr %>%
data.frame %>%
recipe() %>%
step_impute_mean(unit_price, unit_cost) %>%
prep() %>%
juice() %>%
head
# filter(unit_cost %>% is.na)

### R-082:
単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
pr %>%
data.frame %>%
recipe() %>%
step_impute_median(unit_price, unit_cost) %>%
prep() %>%
juice() %>%
#head
filter(unit_cost %>% is.na) %>%head

In [ ]:
pr %>%
  replace_na(list(unit_price = mean(.$unit_price, na.rm = T) %>% round, unit_cost = mean(.$unit_cost, na.rm = T) %>% round))

### R-083:
単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
pr %>%
group_by(category_small_cd) %>%
summarise(
		price_median = unit_price %>% median(na.rm = TRUE) %>% round(),
		cost_median = unit_cost %>% median(na.rm = TRUE) %>% round()
		) %>%
inner_join(pr, by='category_small_cd') %>%
mutate(
	unit_price = ifelse(unit_price %>% is.na, price_median, unit_price),
	unit_cost = ifelse(unit_cost %>% is.na, cost_median, unit_cost)
)

### R-084:
顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [ ]:
library(lubridate)
cu = tbl(con, 'customer')

re %>%
data.frame %>%
mutate(year = sales_ymd %>% ymd %>% year) %>%
group_by(customer_id) %>%
summarize(
	amount_all = sum(amount, na.rm = TRUE),
	amount_2019 = sum(ifelse(year == 2019,amount,0 ), na.rm = TRUE)
) %>%
right_join(cu %>% data.frame, by = 'customer_id') %>%
replace_na(list(amount_all = 0, amount_2019 = 0)) %>%
filter(amount_2019 > 0) %>% head(10)

### R-085:
顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

### R-086:
085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

### R-087:
顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

### R-088:
087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
- 重複していない顧客：顧客ID（customer_id）を設定
- 重複している顧客：前設問で抽出したレコードの顧客IDを設定

顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

### R-089:
売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

### R-090:
レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。